In [1]:
grid = '..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..'
grid2 = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'

In [56]:
rows = 'ABCDEFGHI'
cols = '123456789'

def cross(a, b):
    return [s+t for s in a for t in b]


boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
diag_1 = [[rows[i] + cols[i]  for i in range(9)]]
diag_2 = [[rows[i] + cols[8-i]  for i in range(9)]]
unitlist = row_units + column_units + square_units + diag_1 + diag_2
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

In [60]:
unitlist

[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
 ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'],
 ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
 ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'],
 ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'],
 ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'],
 ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9'],
 ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9'],
 ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9'],
 ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8'],
 ['A9', 'B9', 'C9', 'D9', 'E9',

In [59]:
units['A1']

[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
 ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
 ['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3'],
 ['A1', 'B2', 'C3', 'D4', 'E5', 'F6', 'G7', 'H8', 'I9']]

In [57]:
peers['A1']

{'A2',
 'A3',
 'A4',
 'A5',
 'A6',
 'A7',
 'A8',
 'A9',
 'B1',
 'B2',
 'B3',
 'C1',
 'C2',
 'C3',
 'D1',
 'D4',
 'E1',
 'E5',
 'F1',
 'F6',
 'G1',
 'G7',
 'H1',
 'H8',
 'I1',
 'I9'}

In [4]:
def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

In [5]:
def grid_values(grid, miss_v):
    """Convert grid string into {<box>: <value>} dict with '123456789' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '123456789' if it is empty.
    """
    values = [miss_v if s == '.' else s for s in grid ]
    return dict(zip(boxes, values))

In [6]:
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    for box in values.keys():
        if len(values[box]) == 1:
            delete_v = values[box]
            for peer in peers[box]:
                values[peer] = values[peer].replace(delete_v, '')
    return values

In [13]:
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

In [7]:
def reduce_puzzle(values):
    stalled = False
    while not stalled:
        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

        # Your code here: Use the Eliminate Strategy
        values = eliminate(values)
        # Your code here: Use the Only Choice Strategy
        values = only_choice(values)
        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [8]:
def search(values):
    "Using depth-first search and propagation, try all possible values."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    if values is False:
        return False ## Failed earlier
    if all(len(values[s]) == 1 for s in boxes): 
        return values ## Solved!
    # Choose one of the unfilled squares with the fewest possibilities
    n,s = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    
    # min([(len(sudoku[i]), i) for i in boxes if len(sudoku[i]) > 1])
    
    # Now use recurrence to solve each one of the resulting sudokus, and 
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt

In [9]:
v_for_miss = '.'

display(grid_values(grid, v_for_miss))

. . 3 |. 2 . |6 . . 
9 . . |3 . 5 |. . 1 
. . 1 |8 . 6 |4 . . 
------+------+------
. . 8 |1 . 2 |9 . . 
7 . . |. . . |. . 8 
. . 6 |7 . 8 |2 . . 
------+------+------
. . 2 |6 . 9 |5 . . 
8 . . |2 . 3 |. . 9 
. . 5 |. 1 . |3 . . 


In [10]:
v_for_miss = '123456789'

display(grid_values(grid, v_for_miss))

123456789 123456789     3     |123456789     2     123456789 |    6     123456789 123456789 
    9     123456789 123456789 |    3     123456789     5     |123456789 123456789     1     
123456789 123456789     1     |    8     123456789     6     |    4     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     8     |    1     123456789     2     |    9     123456789 123456789 
    7     123456789 123456789 |123456789 123456789 123456789 |123456789 123456789     8     
123456789 123456789     6     |    7     123456789     8     |    2     123456789 123456789 
------------------------------+------------------------------+------------------------------
123456789 123456789     2     |    6     123456789     9     |    5     123456789 123456789 
    8     123456789 123456789 |    2     123456789     3     |123456789 123456789     9     
123456789 123456789     5     |123456789     1     123456789 |    3   

In [11]:
sudoku = grid_values(grid2, v_for_miss)
sudoku

{'A1': '4',
 'A2': '123456789',
 'A3': '123456789',
 'A4': '123456789',
 'A5': '123456789',
 'A6': '123456789',
 'A7': '8',
 'A8': '123456789',
 'A9': '5',
 'B1': '123456789',
 'B2': '3',
 'B3': '123456789',
 'B4': '123456789',
 'B5': '123456789',
 'B6': '123456789',
 'B7': '123456789',
 'B8': '123456789',
 'B9': '123456789',
 'C1': '123456789',
 'C2': '123456789',
 'C3': '123456789',
 'C4': '7',
 'C5': '123456789',
 'C6': '123456789',
 'C7': '123456789',
 'C8': '123456789',
 'C9': '123456789',
 'D1': '123456789',
 'D2': '2',
 'D3': '123456789',
 'D4': '123456789',
 'D5': '123456789',
 'D6': '123456789',
 'D7': '123456789',
 'D8': '6',
 'D9': '123456789',
 'E1': '123456789',
 'E2': '123456789',
 'E3': '123456789',
 'E4': '123456789',
 'E5': '8',
 'E6': '123456789',
 'E7': '4',
 'E8': '123456789',
 'E9': '123456789',
 'F1': '123456789',
 'F2': '123456789',
 'F3': '123456789',
 'F4': '123456789',
 'F5': '1',
 'F6': '123456789',
 'F7': '123456789',
 'F8': '123456789',
 'F9': '123456789',


In [14]:
display(reduce_puzzle(sudoku))

   4      1679   12679  |  139     2369    269   |   8      1239     5    
 26789     3    1256789 | 14589   24569   245689 | 12679    1249   124679 
  2689   15689   125689 |   7     234569  245689 | 12369   12349   123469 
------------------------+------------------------+------------------------
  3789     2     15789  |  3459   34579    4579  | 13579     6     13789  
  3679   15679   15679  |  359      8     25679  |   4     12359   12379  
 36789     4     56789  |  359      1     25679  | 23579   23589   23789  
------------------------+------------------------+------------------------
  289      89     289   |   6      459      3    |  1259     7     12489  
   5      6789     3    |   2      479      1    |   69     489     4689  
   1      6789     4    |  589     579     5789  | 23569   23589   23689  


In [17]:
for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

IndentationError: unindent does not match any outer indentation level (<ipython-input-17-9c75ba8c1e78>, line 6)

In [30]:
[sudoku[box] for box in unitlist[0] if len(sudoku[box]) >= 2] 

['1679', '12679', '139', '2369', '269', '1239']

In [40]:
test = ['25', '5', '34', '567', '763', '4', '6', '34', '4562']

In [41]:
subset = [test[v] for v in range(len(test)) if len(test[v])>=2]
if len(subset)>=2:
    subset = [test[v] for v in range(len(test)) if len(test[v])==2]
    


In [62]:
sudoku['A1']

'4'

In [74]:
print('dd = ' + str([twin for twin in test if sum([twin==v for v in test]) == 2]))

dd = ['34', '34']


In [79]:
s = [twin for twin in test if sum([twin==v for v in test]) == 2]
s

['34', '34']

In [83]:
my.append('A')
my

['A']

In [116]:
s

['34', '34']

In [101]:
'34' in my

True

In [119]:
my = []

In [117]:
[my.append(v) or my for v in s if v not in my]

[['34']]

In [122]:
[my + [v] for v in s if [v] not in my]

[['34'], ['34']]

In [48]:
for v in range(len(subset)-1):
    if sum([subset[i]==subset[v] for i in range(1, len(subset))]) == 2:
        print(subset[v])

34


In [ ]:
    for unit in unitlist:
        my_subset = [values[box] for box in unit if len(values[box])>=2]
        if len(my_subset)>=2:
            double_digit = [my_subset[box] for box in unit if len(my_subset[box])==2]
            print(double_digit)
            
    return values


In [64]:
my_subset = [sudoku[box] for box in unitlist[0] if len(sudoku[box])>=2]

In [65]:
my_subset

['1679', '12679', '139', '2369', '269', '1239']

In [70]:
[i for i in my_subset if len(i)==3]

['139', '269']

In [ ]:
if digit in values[box]

In [15]:
display(search(sudoku))

4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |9 4 7 
9 5 8 |7 2 4 |3 1 6 
------+------+------
8 2 5 |4 3 7 |1 6 9 
7 9 1 |5 8 6 |4 3 2 
3 4 6 |9 1 2 |7 5 8 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 7 3 |2 9 1 |6 8 4 
1 6 4 |8 7 5 |2 9 3 
